# Coherence of Formation

<a target="_blank" href="https://colab.research.google.com/github/numqi/numqi/blob/main/docs/application/coherence/cof.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

reference: Quantum Coherence and Intrinsic Randomness [arxiv-link](https://arxiv.org/abs/1605.07818)

In [ ]:
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

try:
    import numqi
except ImportError:
    %pip install numqi
    import numqi

np_rng = np.random.default_rng()


## Pure state

For pure state $|\psi\rangle$, the Coherence of Formation is defined as

$$ \Delta(\rho)=\sum_{i}\rho_{ii}|i\rangle\langle i| $$

$$ C_{f}\left(|\psi\rangle\right)=S\left(\Delta\left(|\psi\rangle\langle\psi|\right)\right) $$

Below, we try to calculate the Coherence of Formation for some pure states.

In [ ]:
psi = np.array([1,0,0,0], dtype=np.float64)
print('computational basis: ', numqi.coherence.get_coherence_of_formation_pure(psi))
# TODO for some random pure state

psi = numqi.random.rand_haar_state(4)
print('random pure state: ', numqi.coherence.get_coherence_of_formation_pure(psi))

psi = np.array([1,1,1,1], dtype=np.float64)/2
print('maximally coherent state: ', numqi.coherence.get_coherence_of_formation_pure(psi))

## Mixed state


For mixed state $\rho$, the Coherence of Formation is defined via convex roof extension [arxiv-link](https://arxiv.org/abs/2406.19683)

$$ \begin{align*}
    C_{f}\left(\rho\right)&=\min_{\left\{ p_{\alpha},|\psi_{\alpha}\rangle\right\} }\sum_{\alpha}p_{\alpha}C_{f}\left(|\psi_{\alpha}\rangle\right)\\
    &=\min_{\left\{ \tilde{\rho}_{\alpha}\right\} }\left(\sum_{\alpha}p_{\alpha}\ln p_{\alpha}-\sum_{\alpha,i}\tilde{\rho}_{\alpha,ii}\ln\tilde{\rho}_{\alpha,ii}\right)
\end{align*} $$

where the minimization is over all ensemble decomposition of $\rho=\sum_{\alpha}p_{\alpha}|\psi_{\alpha}\rangle\langle\psi_{\alpha}|$.

In [ ]:
tmp0 = numqi.random.rand_haar_state(4)
dm_target = tmp0.reshape(-1,1) * tmp0.conj()
dim = dm_target.shape[0]
alpha_list = np.linspace(0, 1, 50)

model = numqi.coherence.CoherenceFormationModel(dim, num_term=3*dim)
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-10, print_every_round=0)
cof_list = []
for alpha_i in tqdm(alpha_list):
    model.set_density_matrix(numqi.entangle.hf_interpolate_dm(dm_target, alpha=alpha_i))
    cof_list.append(numqi.optimize.minimize(model, **kwargs).fun)
cof_list = np.array(cof_list)

fig,ax = plt.subplots()
ax.plot(alpha_list, cof_list)
ax.set_xlabel('alpha')
ax.set_ylabel('CoF')
fig.tight_layout()